In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#split the data into test and train 
#use the file feature_engineered.csv
from sklearn.model_selection import train_test_split


In [ ]:
def modAUClogReg(X_train, Y_train, X_test, Y_test):
    '''
    trains a linear regression model on train set and returns AUC on test set
    '''
    #Using Scikit-learn the logistic regression model is built with two easy steps.
    clf_LogReg = LogisticRegression(C = 1e30)
    clf_LogReg.fit(X_train, Y_train)
    Y_pred_lg = clf_LogReg.predict_proba(X_test)[:,1]
    fpr_lg, tpr_lg, thresholds_lg = roc_curve(Y_test, Y_pred_lg, drop_intermediate = True)
    return fpr_lg, tpr_lg, thresholds_lg

#fpr_lg, tpr_lg, thresholds_lg = modAUClogReg(X_train, Y_train, X_test, Y_test)
#roc_auc_lg = auc(fpr_lg, tpr_lg)

In [ ]:
def modAUCSVC(X_train, Y_train, X_test, Y_test):
    '''
    trains a SVC on train set and returns AUC on test set
    '''
    clf_SVC = SVC(kernel = "linear", probability = True)
    clf_SVC.fit(X_train, Y_train)
    Y_pred_SVC = clf_SVC.predict_proba(X_test)[:,1]
    fpr_svc, tpr_svc, thresholds_svc = roc_curve(Y_test, Y_pred_SVC)
    roc_auc_svc = auc(fpr_svc, tpr_svc)
    return fpr_svc, tpr_svc, thresholds_svc, roc_auc_svc
#fpr_svc, tpr_svc, thresholds_svc, roc_auc_svc = modAUCSVC(X_train, Y_train, X_test, Y_test)
#roc_auc_svc = auc(fpr_svc, tpr_lg)

In [ ]:
#import staments making our work easier
from matplotlib import pyplot as plt
#ploting of the curves
plt.title('Receiver Operating Characteristic')

plt.plot(fpr_lg, tpr_lg, 'b', label = 'AUC Log Reg = %0.2f' % roc_auc_lg)

plt.plot(fpr_svc,tpr_svc,'g',label ='AUC SVM = %0.2f' %roc_auc_svc)

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')



ax2 = plt.gca().twinx()
ax2.plot(fpr_lg, thresholds_lg, markeredgecolor='r', color='r')
ax2.set_ylabel('Threshold',color='r')
ax2.set_ylim([thresholds_lg[-1],thresholds_lg[0]])
ax2.set_xlim([fpr_lg[0],fpr_lg[-1]])

plt.show()

In [ ]:
def xValSVM(dataset, label_name, k, cs):
    
    #initialize a dictionary for storing auc scores
    aucs = dict()
    
    #Get the indices for kfold cross validation
    kf = KFold(n_splits=k, shuffle = True)
    train_indices = list()
    test_indices = list()
    
    #initialize a dictionary to store values with respect to c different values of k
    for c in cs:
        aucs[c] = list()
             
    for train_index, test_index in kf.split(dataset):
        train_indices.append(train_index)
        test_indices.append(test_index)
    #print(test_indices, train_indices)
    
    #perform KFold for different values of c
    for f in range(k):
        
        train_index = list(train_indices[f])
        test_index = list(test_indices[f])
        X_train = dataset.iloc[train_index].drop(target,1)
        Y_train = dataset.iloc[train_index][target]
        
        X_test = dataset.iloc[test_index].drop(target,1)
        Y_test = dataset.iloc[test_index][target]
        
        for c in cs:
            SVMCrss = SVC(C = c , kernel = "linear",probability = True)
            SVMCrss.fit(X_train, Y_train)
            pred = SVMCrss.predict_proba(X_test)[:,1]
            fpr,tpr,threshold = roc_curve(Y_test, pred)
            roc_auc = auc(fpr, tpr)
            aucs[c].append(roc_auc)
        
    return aucs

In [ ]:
stats = dict()
#Intialize a dictionary for storing statistics, though it is not a very good idea when you have a lot of keys
for c in C:
    stats[c] = list()
maximum = 0
for c in C:
    #First value with respect to a key is mean
    stats[c].append(np.mean(aucs[c]))
    #Second value with respect to a key is standard deviation
    stats[c].append(np.std(aucs[c]))
    #Second value with respect to a key is mean - standard deviation
    stats[c].append(stats[c][0]-stats[c][1])
    ##Third value with respect to a key is mean + standard deviation
    stats[c].append(stats[c][0]+stats[c][1])
    #Find the maximum mean - standard deviation
    if(maximum < stats[c][2]):
        maximum = stats[c][2]
        VariableC = c
print("The value of maximum mean - stderror is",maximum )
print("The value of C corresponding to this is", VariableC)